### 5th Process:

- Merge all the stats from batters and pitchers to play data
- To predict the event in a match

### Import

In [2]:
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', None)

In [4]:
all_player_stats = pd.read_csv('all_player_stats.csv')
top_players_plays = pd.read_csv('mlb_top_players_plays.csv')
top_players_id =pd.read_csv('mlb_top_players_id.csv')

In [10]:
valid_ids = set(all_player_stats['player_id'])

top_players_plays = (
    top_players_plays[
        (top_players_plays['batter_id'].isin(valid_ids)) &
        (top_players_plays['pitcher_id'].isin(valid_ids))
        ]
    )

In [11]:
batter_stats = all_player_stats.copy().add_prefix("b_")
pitcher_stats = all_player_stats.copy().add_prefix("p_")
batter_stats = batter_stats.rename(columns={'b_player_id': 'batter_id', 'b_season': 'season'})
pitcher_stats = pitcher_stats.rename(columns={'p_player_id': 'pitcher_id', 'p_season': 'season'})

In [12]:
merged_data = (
    top_players_plays
    .filter([
        'season', 'batter_id', 'pitcher_id',
        'event_type', 'vs_RHB', 'vs_LHB',
        'vs_SHB', 'vs_RHP', 'vs_LHP'
    ])
    .merge(batter_stats, on=['season', 'batter_id'], how='left')
    .merge(pitcher_stats, on=['season', 'pitcher_id'], how='left')
)

In [21]:
null_percentage = merged_data.isnull().mean() * 100
merged_data = merged_data.loc[:, null_percentage <= 65]

In [24]:
merged_data = merged_data.drop(columns=['batter_id', 'pitcher_id', 'season'])

In [25]:
merged_data.to_csv('all_plays_merged_data.csv', index=False)